In [13]:
import pandas as pd
import numpy as np
import joblib

from sklearn.metrics import accuracy_score, classification_report

In [14]:
    # LOAD TEST DATA
    test_df = pd.read_csv("../data/raw/test.csv")
    
    X_test_raw = test_df.drop(columns=["Activity", "subject"])
    y_test_str = test_df["Activity"]
    
    print("Test raw shape:", X_test_raw.shape)
    print("Test labels shape:", y_test_str.shape)

Test raw shape: (2947, 561)
Test labels shape: (2947,)


In [15]:
# LOAD SAVED ARTIFACTS
model = joblib.load("../model/extra_trees_model.pkl")
scaler = joblib.load("../model/scaler.pkl")
selected_feature_indices = joblib.load("../model/selected_features.pkl")

print("Model loaded:", type(model))
print("Selected features:", len(selected_feature_indices))

Model loaded: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>
Selected features: 150


In [16]:
def engineer_features(X, scaler):
    # Scaling
    X_scaled = pd.DataFrame(
        scaler.transform(X),
        columns=X.columns
    )

    # Statistical features
    X_stats = pd.DataFrame({
        "row_mean": X.mean(axis=1),
        "row_std": X.std(axis=1),
        "row_max": X.max(axis=1),
        "row_min": X.min(axis=1)
    })

    # Energy features
    energy_cols = [c for c in X.columns if "energy()" in c]
    X_energy = pd.DataFrame({
        "energy_mean": X[energy_cols].mean(axis=1),
        "energy_std": X[energy_cols].std(axis=1)
    })

    # Angle features
    angle_cols = [c for c in X.columns if c.startswith("angle")]
    X_angles = X[angle_cols]

    # Final engineered matrix
    X_engineered = pd.concat(
        [X_scaled, X_stats, X_energy, X_angles],
        axis=1
    )

    return X_engineered

In [17]:
# FEATURE ENGINEERING
X_test_engineered = engineer_features(X_test_raw, scaler)

print("Engineered test shape:", X_test_engineered.shape)

# FEATURE SELECTION (same indices as training)
X_test = X_test_engineered.iloc[:, selected_feature_indices]

print("Final test feature shape:", X_test.shape)

Engineered test shape: (2947, 574)
Final test feature shape: (2947, 150)


In [18]:
label_map = {
    "LAYING": 0,
    "SITTING": 1,
    "STANDING": 2,
    "WALKING": 3,
    "WALKING_DOWNSTAIRS": 4,
    "WALKING_UPSTAIRS": 5
}

y_test_enc = y_test_str.map(label_map).values

In [19]:
# PREDICTION
y_pred_enc = model.predict(X_test)

# METRICS
print("\n✅ FINAL TEST ACCURACY:")
print(accuracy_score(y_test_enc, y_pred_enc))

print("\n📊 CLASSIFICATION REPORT:\n")
print(
    classification_report(
        y_test_enc,
        y_pred_enc,
        target_names=list(label_map.keys()),
        zero_division=0
    )
)


✅ FINAL TEST ACCURACY:
0.9518154054971157

📊 CLASSIFICATION REPORT:

                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       537
           SITTING       0.96      0.90      0.93       491
          STANDING       0.92      0.97      0.94       532
           WALKING       0.94      0.99      0.97       496
WALKING_DOWNSTAIRS       0.98      0.90      0.93       420
  WALKING_UPSTAIRS       0.92      0.94      0.93       471

          accuracy                           0.95      2947
         macro avg       0.95      0.95      0.95      2947
      weighted avg       0.95      0.95      0.95      2947

